In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import warnings
warnings.filterwarnings('ignore')

In [2]:
crsp = pd.read_csv(r'C:\Users\edgar\Downloads\crsp_top1000.csv', sep='\t')
FF = pd.read_csv(r'C:\Users\edgar\Downloads\FF.csv', sep='\t')
ratios = pd.read_csv(r'C:\Users\edgar\Downloads\financialratios.csv', sep='\t')
Ratios_Manual = pd.read_excel(r'C:\Users\edgar\Downloads\FinancialRatiosManual.xlsx')

In [3]:
FF['date'] = pd.to_datetime(FF['date'])
Ratios_Manual['Financial Ratio'] = Ratios_Manual['Financial Ratio'].str.replace('\n',' ')
crsp['date'] = pd.to_datetime(crsp['date'])
ratios['public_date'] = pd.to_datetime(ratios['public_date'])
df_all = pd.merge_asof(ratios.sort_values('public_date'), crsp.sort_values('date'), 
                           by='permno', 
                           left_on='public_date', 
                           right_on='date',
                           tolerance=pd.Timedelta(3, 'days'))
df_all = df_all.sort_values(by = ['permno', 'date'])

In [72]:
df_shift = df_all[['permno','date','month','ret']]
L = []
for i in range(1,len(df_shift)) :
    if df_shift.iloc[i, 0] == df_shift.iloc[i-1, 0] :
        if (df_shift.iloc[i, 2] == df_shift.iloc[i-1, 2] + 1) or (df_shift.iloc[i, 2] == 1 and df_shift.iloc[i-1, 2] == 12) :
            L.append(df_shift.iloc[i, 3])
        else :
            L.append(np.nan)
    else :
        L.append(np.nan)

L = L + [np.nan]

df_all['Ret t+1'] = L

In [5]:
df_all_insample = df_all[df_all['public_date'] < '2017-01-01'].dropna(subset = ['prc'])
df_all_outsample = df_all[df_all['public_date'] >= '2017-01-01'].dropna(subset = ['prc'])

In [6]:
def Listret(fratio,df,quant) :
    Date = list(df.sort_values('date')['date'].drop_duplicates())
    ListRet = []
    for i in range(len(Date)-1) :
        test = df[df["date"] == Date[i]][['permno',fratio]].sort_values(fratio).dropna()
        testbottom20 = test[test[fratio] < test.quantile(quant)[fratio]]
        testtop20 = test[test[fratio] > test.quantile(1-quant)[fratio]]
        df_all_test_top = df[df["date"] == Date[i+1]][df["permno"].isin(list(testtop20['permno']))]
        df_all_test_bottom = df[df["date"] == Date[i+1]][df["permno"].isin(list(testbottom20['permno']))]
        ListRet.append((sum([float(x) for x in list(df_all_test_top['ret'])]) - sum([float(x) for x in list(df_all_test_bottom['ret'])]))/(len(df_all_test_top) + len(df_all_test_bottom))) 
    return(ListRet)

In [7]:
def ListretMultiple(List_fratio,df,quant) :
    Date = list(df.sort_values('date')['date'].drop_duplicates())
    ListRet = []
    for i in range(len(Date)-1) :
        ListFirmTop = []
        ListFirmBottom = []
        for ratio in List_fratio :
            test = df[df["date"] == Date[i]][['permno',ratio]].sort_values(ratio).dropna()
            testbottom = test[test[ratio] < test.quantile(quant)[ratio]]
            testtop = test[test[ratio] > test.quantile(1-quant)[ratio]]
            ListFirmBottom += list(testbottom['permno'])
            ListFirmTop += list(testtop['permno'])
        df_all_test_top = df[df["date"] == Date[i+1]][df["permno"].isin(ListFirmTop)]
        df_all_test_bottom = df[df["date"] == Date[i+1]][df["permno"].isin(ListFirmBottom)]
        ListRet.append((sum([float(x) for x in list(df_all_test_top['ret'])]) - sum([float(x) for x in list(df_all_test_bottom['ret'])]))/(len(df_all_test_top) + len(df_all_test_bottom))) 
    return(ListRet)

In [8]:
def Listret_Industry(fratio,df) :
    Date = list(df.sort_values('date')['date'].drop_duplicates())
    ListRet = []
    for i in range(len(Date)-1) :
        test = df[df["date"] == Date[i]][['permno','sic2',fratio]].sort_values(fratio).dropna()
        List_Industry = list(test['sic2'].drop_duplicates())
        List_Long = []
        List_Short = []
        for ind in List_Industry :
            testind = test[test['sic2'] == ind]
            testbottom20 = testind[testind[fratio] < testind.quantile(.2)[fratio]][testind[fratio] > testind.quantile(.05)[fratio]]
            List_Short += list(testbottom20['permno'].drop_duplicates())
            testtop20 = testind[testind[fratio] > testind.quantile(.8)[fratio]][testind[fratio] < testind.quantile(.95)[fratio]]
            List_Long += list(testtop20['permno'].drop_duplicates())
        df_all_test_top = df[df["date"] == Date[i+1]][df["permno"].isin(List_Long)]
        df_all_test_bottom = df[df["date"] == Date[i+1]][df["permno"].isin(List_Short)]
        ListRet.append((sum([float(x) for x in list(df_all_test_top['ret'])]) - sum([float(x) for x in list(df_all_test_bottom['ret'])]))/(len(df_all_test_top) + len(df_all_test_bottom))) 
    return(ListRet)

In [9]:
def Regress(df,ListRet) :
    df_regress = pd.DataFrame({'date' : list(df.sort_values('date')['date'].drop_duplicates())[1:], 'Return Portfolio' : ListRet}).merge(FF, how='left', on='date')
    df_regress['R_Rf'] = df_regress['Return Portfolio'] - df_regress['rf']  
    return(df_regress)

In [18]:
def DF_Recap_Sum(x,df,quant) :
    try: result = sum(Listret(x,df,quant)) 
    except: result =  np.nan
    return result

def DF_Recap_Var(x,df,quant) :
    try: result = np.var(Listret(x,df,quant)) 
    except: result =  np.nan
    return result

def DF_Recap_Alpha(x,df,quant) :
    try: 
        df_reg = Regress(df,Listret(x,df,quant))
        regress = sm.ols(formula="R_Rf ~ mktrf+smb+hml+qual+umd", data=df_reg).fit()
        result = regress.params[0]
    except: result =  np.nan
    return result

def DF_Recap_Sharpe(x,df,quant) :
    try: 
        df_reg = Regress(df,Listret(x,df,quant))
        result = np.sqrt(12)*df_reg['R_Rf'].mean()/df_reg['R_Rf'].std()
    except: result =  np.nan
    return result

In [16]:
df1 = df_all_insample
df2 = df_all_outsample
quant = 0.05

In [20]:
Recap = pd.DataFrame({'Ratios' : list(Ratios_Manual['Variable Name'])})
Recap['Return In Sample'] = Recap['Ratios'].apply(lambda x: DF_Recap_Sum(x,df1,quant))
Recap['Return Out Sample'] = Recap['Ratios'].apply(lambda x: DF_Recap_Sum(x,df2,quant))
Recap['Variance In Sample'] = Recap['Ratios'].apply(lambda x: DF_Recap_Var(x,df1,quant))
Recap['Variance Out Sample'] = Recap['Ratios'].apply(lambda x: DF_Recap_Var(x,df2,quant))
Recap['Sharpe Ratio'] = Recap['Ratios'].apply(lambda x: DF_Recap_Sharpe(x,df2,quant))
Recap['Alpha'] = Recap['Ratios'].apply(lambda x: DF_Recap_Alpha(x,df2,quant))
Recap

,Ratios,Return In Sample,Return Out Sample,Variance In Sample,Variance Out Sample,Sharpe Ratio,Alpha
0,capital_ratio,3.189712,0.211564,0.002767,0.000673,0.271081,-0.000583
1,equity_invcap,-2.247823,-0.014776,0.002686,0.000530,-0.170623,0.000849
2,debt_invcap,3.112235,0.135120,0.002999,0.000709,0.125616,-0.001664
3,totdebt_invcap,1.835225,0.031683,0.002199,0.000489,-0.074121,-0.001565
4,at_turn,1.136155,0.230045,0.000640,0.000518,0.350487,-0.001077
...,...,...,...,...,...,...,...
67,pe_inc,0.157866,0.018301,0.000219,0.000159,-0.180322,-0.000676
68,pe_op_basic,-0.289427,0.031500,0.000287,0.000215,-0.112809,-0.000662
69,pe_op_dil,NaN,0.000304,NaN,0.000216,-0.214931,-0.001118
70,ps,-1.221323,-0.342099,0.001857,0.001619,-0.493053,-0.004843


In [53]:
Recap.sort_values('Sharpe Ratio',ascending=False)

,Ratios,Return In Sample,Return Out Sample,Variance In Sample,Variance Out Sample,Sharpe Ratio,Alpha
9,sale_invcap,1.888746,0.521820,0.001057,0.000344,1.189280,0.003446
8,sale_equity,1.545217,0.552389,0.001235,0.000435,1.128411,0.003783
65,pcf,0.339741,0.395566,0.000369,0.000300,0.926857,0.003105
36,GProf,1.263137,0.390655,0.000847,0.000459,0.739067,0.000035
38,aftret_equity,0.865529,0.281866,0.000932,0.000228,0.690084,0.000976
...,...,...,...,...,...,...,...
32,RD_SALE,NaN,NaN,NaN,NaN,NaN,NaN
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,PEG_1yrforward,NaN,NaN,NaN,NaN,NaN,NaN
59,PEG_ltgforward,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
Recap_Pos = Recap[Recap['Return In Sample'] > 0][Recap['Sharpe Ratio'] > 0.2].sort_values('Return In Sample',ascending=False)

In [48]:
ListRatios = list(Recap_Pos['Ratios'])
ListSelect = [ListRatios[0]]
ListRet0 = ListretMultiple(ListSelect,df1,quant)
A = 5
U = np.mean(ListRet0) -1/2*A*np.var(ListRet0)
for x in ListRatios[1:] :
    ListTest = ListSelect + [x]
    ListRet1 = ListretMultiple(ListSelect,df1,quant)
    UTest = np.mean(ListRet1) -1/2*A*np.var(ListRet1)
    if UTest > U :
        ListSelect = ListTest
        U = UTest
print(ListSelect)
print(U)

['cash_debt']
0.0037631940755717187


In [51]:
ListRatios = list(Recap_Pos['Ratios'])
ListSelect = [ListRatios[0]]
ListRet0 = ListretMultiple(ListSelect,df1,quant)
print(np.mean(ListRet0)) 
print(np.var(ListRet0))

0.009875270098970238
0.0027667362860926137


In [21]:
ratios[list(Recap.sort_values('Return Out Sample',ascending=False).dropna().head(5)['Ratios'])].corr()

,invt_act,dpr,staff_sale,sale_equity,sale_invcap
invt_act,1.000000,0.001711,-0.008543,0.015045,0.148594
dpr,0.001711,1.000000,-0.000673,0.000070,0.001491
staff_sale,-0.008543,-0.000673,1.000000,-0.000072,-0.002913
sale_equity,0.015045,0.000070,-0.000072,1.000000,0.023578
sale_invcap,0.148594,0.001491,-0.002913,0.023578,1.000000


In [25]:
quant = 0.05
List_fratio = ['roa','roe','evm']
ListRet1 = ListretMultiple(List_fratio,df1,quant)
ListRet2 = ListretMultiple(List_fratio,df2,quant)
df_reg = Regress(df2,ListRet2)
result = sm.ols(formula="R_Rf ~ mktrf+smb+hml+qual+umd", data=df_reg).fit()
print('Return In Sample = ' + str(sum(ListRet1)))
print('Return Out Sample = ' + str(sum(ListRet2)))
print('Variance In Sample = ' + str(np.var(ListRet1)))
print('Variance Out Sample = ' + str(np.var(ListRet2)))
print('Alpha = ' + str(result.params[0]))
print('Sharpe Ratio = ' + str(np.sqrt(12)*df_reg['R_Rf'].mean()/df_reg['R_Rf'].std()))

Return In Sample = 1.0483104290736003
Return Out Sample = 0.4046998229796052
Variance In Sample = 0.00037334248067828914
Variance Out Sample = 0.00026799842864357824
Alpha = 0.0009110396626247776
Sharpe Ratio = 1.00098852409567


In [13]:
ListRet1 = Listret_Industry(fratio,df1)
ListRet2 = Listret_Industry(fratio,df2)
df_reg = Regress(df2,ListRet2)
result = sm.ols(formula="R_Rf ~ mktrf+smb+hml+qual+umd", data=df_reg).fit()
print('Return In Sample = ' + str(sum(ListRet1)))
print('Return Out Sample = ' + str(sum(ListRet2)))
print('Alpha = ' + str(result.params[0]))
print('Sharpe Ratio = ' + str(np.sqrt(12)*df_reg['R_Rf'].mean()/df_reg['R_Rf'].std()))

Return In Sample = 0.526788865222833
Return Out Sample = 0.09438668183337642
Alpha = -0.0012044196857463564
Sharpe Ratio = 0.135834700292358
